In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch
from torch.distributions.normal import Normal

from torch.distributions.multivariate_normal import MultivariateNormal

In [3]:
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
length = 10

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial State
        self.s_0 = nn.Parameter(torch.tensor([5.525, 5.1, 5.1,5.1], requires_grad=True)).requires_grad_(True)
        
        self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
        self.mu1 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        
        self.mu2 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        
        self.std1 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        self.std2 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        

        
        
        

    def forward(self, x,t,pred):

        x = F.softmax((x.clone()),dim=0)
        l=0.0
        std1=torch.exp(self.std1).clip(1e-16,1)
        std2=torch.exp(self.std2).clip(1e-16,1)
        

        x_temp=x.clone()
            
            
            
            
        dist=MultivariateNormal(self.mu1[0]+torch.exp(self.mu2[0]*(torch.tensor(0))), (std1[0])*torch.eye(length).to(device))
        
        o_1 = torch.exp(dist.log_prob(pred[0]))

        x_temp[0]=o_1*x[0].clone()

        dist=MultivariateNormal(self.mu1[1]+torch.exp(self.mu2[1]*(torch.tensor(0))),( std1[1])*torch.eye(length).to(device))

        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[1]=o_1.clone()*x[1].clone()

        dist=MultivariateNormal(self.mu1[2]+torch.exp(self.mu2[2]*(torch.tensor(0))), (std1[2])*torch.eye(length).to(device))
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[2]=o_1*x[2].clone()
        
        dist=MultivariateNormal(self.mu1[3]+torch.exp(self.mu2[3]*(torch.tensor(0))), (std1[3])*torch.eye(length).to(device))
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[3]=o_1*x[3].clone()
        
#         print(x_temp)
        


        x_temp=x_temp.clip(1e-16,1)
        
        x=x_temp.clone()/torch.sum(x_temp.clone())
        l+=torch.log(torch.sum(x_temp.clone()))# -0.01*state
        
        for i in range(1,t): 

            
            x=torch.matmul(x,F.softmax(self.T,dim=1))
            x_temp=x.clone()
            

            x_temp1=x.clone()
            dist=MultivariateNormal(self.mu1[0]+torch.exp(self.mu2[0]*(torch.tensor(i))), (std1[0])*torch.eye(length).to(device))
        
            o_1 = torch.exp(dist.log_prob(pred[i]))

            x_temp[0]=o_1*x[0].clone()

            dist=MultivariateNormal(self.mu1[1]+torch.exp(self.mu2[1]*(torch.tensor(i))), (std1[1])*torch.eye(length).to(device))

            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[1]=o_1.clone()*x[1].clone()

            dist=MultivariateNormal(self.mu1[2]+torch.exp(self.mu2[2]*(torch.tensor(i))),( std1[2])*torch.eye(length).to(device))
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[2]=o_1*x[2].clone()

            dist=MultivariateNormal(self.mu1[3]+torch.exp(self.mu2[3]*(torch.tensor(i))), (std1[3])*torch.eye(length).to(device))
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[3]=o_1*x[3].clone()

            
            
            
#             print(x_temp)
#             print(pred[i])
            x_temp=x_temp.clip(1e-16,1)

            x=x_temp.clone()/torch.sum(x_temp.clone())
#             print(x)
#             print("-----------------------------")
            l+=torch.log(torch.sum(x_temp.clone()))
        return l
    
    def transition(self,x):
        x=torch.matmul(x.to(device),F.softmax(self.T,dim=1))
        x_temp=x.clone()
        return x
    def Observation(self,x):

#         o_1= self.emission((x))
#         o_1[1]=torch.exp(o_1[1])

        return mu
    
    def simulate(self):
         o=[]
         x1 = 0
         for i in range(0,1000):
             print(x1)
            
            
            
            
             if x1==0:
                x=torch.tensor([1.0,0.0,0]).to(device)
             if x1==1:
                x=torch.tensor([0, 1.0,0.0]).to(device)
             if x1==2:
                x=torch.tensor([0.0,0.0,1]).to(device)
             o_1= self.emission((x)) 
             o_1=o_1/torch.sum(o_1)

             o.append(torch.distributions.Categorical(o_1.to(device)).sample())
             if x1==2:
                
                return o
             x = self.transition((x))
             x1=(torch.distributions.Categorical(x.to(device)).sample()).item()



In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'

cuda:0


In [6]:


T_matrix=torch.rand(3, 3).to(device)
T_matrix=torch.tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]], device=device)

# T_matrix[1,1]=8

# T_matrix[2,2]=8

# T_matrix[0,0]=8

T_matrix=T_matrix/torch.sum(T_matrix, 1).reshape(4,1)


s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)
O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_mean1=torch.tensor([0.01,0.02,0.03,0.04]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)






print(T_matrix)

print(s_0)


tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]])
tensor([1., 0., 0.])


In [7]:
def simulate_HMM(T_matrix,O_matrix,x,t):
    x=0
    
    
    t=0
    o=[]
#     o_1=torch.matmul(x.to(device),O_matrix.to(device))
# #         print(o_1)
#     o.append(Normal(O_matrix_mean[x],O_matrix_std[x]).sample().to(device))
    
    K=[2*i/(length) for i in range(1,length+1)]
    K=torch.tensor(K).to(device)
    o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
    t=1
    for i in range(1,1000):
#         print(i)
        x=(torch.distributions.Categorical(T_matrix[x].to(device)).sample())
#         print(x)
        
        
        t+=1
        if x==0:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==1:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(3*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==2:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(torch.exp(O_matrix_mean1[x]*torch.tensor(t)))*torch.sin(6*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==3:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(torch.exp(O_matrix_mean1[x]*torch.tensor(t)))*torch.sin(9*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        t1=t
        if (t==50) :
            break
    
    

    return o,t,t1



In [8]:
o=simulate_HMM(T_matrix,O_matrix_mean,torch.tensor([1.0,0,0.0]),1000)
o

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16260\2232260582.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16260\2232260582.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(3*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Loc

([tensor([ 0.1979,  1.9243,  2.9772,  7.1350,  9.8406,  5.8789,  4.1596,  1.2357,
           0.1411, -0.0480]),
  tensor([ 2.5946,  1.1703,  1.9082,  6.0196,  9.1351,  6.7261,  2.8895,  2.8168,
           0.6430, -0.2222]),
  tensor([-0.0510,  2.1605,  4.0198,  3.0491,  7.7566,  4.9561,  0.1876,  2.9438,
           1.0787,  0.4534]),
  tensor([0.0251, 1.7300, 3.3036, 3.8160, 9.6802, 5.6071, 0.7935, 1.6137, 1.5774,
          0.9456]),
  tensor([ 2.3242,  3.0956,  0.8362,  4.1714, 10.1439,  5.0792,  2.7953,  0.3056,
           2.4936,  0.8940]),
  tensor([-1.1277,  1.8407,  3.7991,  3.6969, 10.2445,  4.5513,  3.9037,  0.1416,
           2.3529,  0.4175]),
  tensor([ 4.8116,  2.0942,  2.7510,  4.6906, 10.0919,  4.8095,  2.4970,  3.1377,
           1.2060,  0.5455]),
  tensor([ 1.1789, -0.1920,  1.5687,  3.6682,  8.9068,  4.7851,  3.0610,  2.1608,
           0.6730,  2.5273]),
  tensor([0.5112, 2.2296, 2.5282, 6.4651, 8.2043, 3.2340, 2.4622, 2.7151, 1.3262,
          0.9175]),
  tensor([ 2

In [9]:
# def distribution_initalize(o):
#     s=torch.zeros(10)
#     for i in range(0,10):
#         s[i]=torch.sum(o==i)
        
#     return s#/torch.sum(s)
# def distribution_update(dist,o):

#     return dist + distribution_initalize(o)


In [10]:
# f=distribution_initalize(o[0])

def Expectation_diff(o_1):
    return torch.sum(o_1[0])


def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=torch.abs(e2-e1)
    return k


In [11]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import time

start_time = time.time() 
for batch_size in [5]:
    trial =0 
    while trial < 5:
        t=50
        
        try: 
            pred1=[]
            for i in range(0,batch_size):
                pred1.append(simulate_HMM(T_matrix,O_matrix_mean,s_0,t))
            import torch.optim as optim
            net = Net()
            net.to(device)
            optimizer = optim.Adam(net.parameters(), lr=.01)
            scheduler =lr_scheduler.LinearLR(optimizer,start_factor=1, end_factor=0.001, total_iters=75)
            net.train()
    
            running_loss = 0.0
            o,t,t__1=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
            # dist=distribution_initalize(o)
            for epoch in range(100):  # loop over the dataset multiple times
    
    
    
    
                # zero the parameter gradients
                optimizer.zero_grad()
    
                # forward + backward + optimize
    
    
    
                loss=0.0
    
                for i in range(0,batch_size):
                    optimizer.zero_grad()
                    loss=0.0
    
            #         o,t=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
                    o,t,t_11=pred1[i]
                    pred=torch.zeros((1,t), dtype=torch.int32)
                    pred=o
            #         dist=distribution_update(dist,o)
                    loss-=net(net.s_0,t,pred)
                   
    
    
                    #+ l1_reg*l1_lambda
                #     print(loss)
                    loss=loss #+ 10*(torch.sum(t1*t2)/((torch.sum(t1**2)**0.5)*torch.sum(t2**2)**0.5) +torch.sum(t3*t2)/((torch.sum(t3**2)**0.5)*torch.sum(t2**2)**0.5)+torch.sum(t1*t3)/((torch.sum(t1**2)**0.5)*torch.sum(t3**2)**0.5) )
                    loss.backward()
                    optimizer.step()
                scheduler.step()
                    # print statistics
            # print(f'[{trial + 1}] loss: {loss/t}')
    
            # print('Finished Training')
    
            torch.save(pred1, "data_normal\ "+str(batch_size)+"_ee_"+ str(trial)+ "_data.pt")
            torch.save(net.state_dict(),  "Model_normal\ " +str(batch_size)+"_ee_"+ str(trial)+ "_model.pth")
            trial+=1
        except:
            print(f'[{trial + 1}] loss: {loss/t}')
            
            
            pass
end_time = time.time()  # Capture end time after the loop completes

print(f"Total execution time: {end_time - start_time} seconds")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16260\2232260582.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16260\2232260582.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local

Total execution time: 758.6820185184479 seconds
